<a href="https://colab.research.google.com/github/BarrenWardo/Py-Books/blob/main/Upload_Helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upload Helper

In [ ]:
# @title Setup

!pip install requests
!apt update
!apt install aria2 -y

# @markdown GoFile Servers List - [Click Here](https://api.gofile.io/servers) \
# @markdown GoFile Auth Token - [Click Here](https://gofile.io/myprofile) \
# @markdown Pixeldrain API Key - [Click Here](https://pixeldrain.com/user/api_keys) \
# @markdown upload_server : ```0 - Both``` | ```1 - GoFile``` | ```2 - Pixeldrain```

In [ ]:
# @title Downloader

URL = "" # @param {type:"string", placeholder:"Download URL"}

!aria2c "$URL" -x16 --summary-interval=10 --seed-time=60

In [ ]:
# @title Uploader

import os
import requests
import time

# User-defined variables
upload_server = 1 # @param [0,1,2] {type:"raw"}
file_path = "" # @param {type:"string", placeholder:"File/path/here.txt"}
delete_after_upload = True  # @param {type:"boolean"}
gofile_server = ""  # @param {type:"string", placeholder:"store5"}
gofile_auth_token = "" # @param {type:"string", placeholder:"gofile_auth_token"}
pixeldrain_auth_token = ""  # @param {type:"string", placeholder:"pixeldrain_api_key"}

# Ensure the file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file '{file_path}' does not exist.")

# Get file size
file_size = os.path.getsize(file_path)

# Helper function for progress tracking
def upload_with_progress(file_path, url, headers=None, auth=None):
    with open(file_path, "rb") as file:
        chunk_size = 8192  # 8 KB
        uploaded_size = 0
        start_time = time.time()

        def progress_callback(chunk):
            nonlocal uploaded_size
            uploaded_size += len(chunk)
            elapsed_time = time.time() - start_time
            speed = uploaded_size / elapsed_time if elapsed_time > 0 else 0
            remaining_time = (file_size - uploaded_size) / speed if speed > 0 else float('inf')

            progress_percentage = (uploaded_size / file_size) * 100
            print(f"Uploaded: {uploaded_size / (1024 * 1024):.2f} MB / {file_size / (1024 * 1024):.2f} MB ({progress_percentage:.2f}%) | Speed: {speed / (1024 * 1024):.2f} MB/s | Remaining Time: {remaining_time:.2f} s", end="\r")

        class FileWithProgress:
            def __init__(self, file):
                self.file = file

            def read(self, size=None):
                chunk = self.file.read(size)
                if chunk:
                    progress_callback(chunk)
                return chunk

            def __getattr__(self, attr):
                return getattr(self.file, attr)

        files = FileWithProgress(file)

        if headers:
            response = requests.post(url, headers=headers, files={"file": (os.path.basename(file_path), files)})
        elif auth:
            headers = {"Content-Type": "application/octet-stream"}
            response = requests.put(url, data=files, headers=headers, auth=auth)
        else:
            raise ValueError("Either headers or auth must be provided.")

        print("\nUpload complete.")
        return response

# Upload to GoFile
if upload_server in (0, 1):
    try:
        print("Uploading to GoFile...")
        headers = {
            "Authorization": f"Bearer {gofile_auth_token}"
        }
        upload_url = f"https://{gofile_server}.gofile.io/contents/uploadfile"

        response = upload_with_progress(file_path, upload_url, headers=headers)
        response_data = response.json()

        if response.status_code == 200 and response_data.get("status") == "ok":
            download_page_url = response_data['data']['downloadPage']
            print(f"GoFile URL: {download_page_url}")
            print("Full Response:", response_data)
        else:
            print("GoFile upload failed.")
            print("Full Response:", response_data)
    except Exception as e:
        print(f"An error occurred while uploading to GoFile: {e}")

# Upload to Pixeldrain
if upload_server in (0, 2):
    try:
        print("\nUploading to Pixeldrain...")
        file_name = os.path.basename(file_path)[:255]  # Limit the file name to 255 characters
        upload_url = f"https://pixeldrain.com/api/file/{file_name}"
        auth = ("api", pixeldrain_auth_token)  # HTTP Basic Authentication

        response = upload_with_progress(file_path, upload_url, auth=auth)

        if response.status_code == 201:
            try:
                response_data = response.json()
                pixeldrain_url = f"https://pixeldrain.com/u/{response_data['id']}"
                print("File uploaded successfully to Pixeldrain!")
                print("Pixeldrain URL:", pixeldrain_url)
                print("Full Response:", response_data)
            except ValueError:
                print("Pixeldrain upload succeeded, but could not parse JSON response.")
                print("Response Content:", response.text)
        else:
            print(f"Pixeldrain upload failed with status code {response.status_code}.")
            print("Full Response:", response.text)
    except Exception as e:
        print(f"An error occurred while uploading to Pixeldrain: {e}")

# Optionally delete the file after upload
if delete_after_upload:
    os.remove(file_path)
    print(f"The file '{file_path}' has been deleted after upload.")